In [9]:
%%writefile line_chart.template
<!DOCTYPE html>
<meta charset="utf-8">
<style>

body {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.line {
  fill: none;
  stroke: steelblue;
  stroke-width: 1.5px;
}

</style>
<body>
<script src="http://d3js.org/d3.v3.min.js" charset="utf-8"></script>
<script>

var margin = {top: 20, right: 80, bottom: 30, left: 50},
    width = 1700 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

var parseDate = d3.time.format("%Y%m%d").parse;

var x = d3.time.scale()
    .range([0, width]);

var y = d3.scale.linear()
    .range([height, 0]);

var color = d3.scale.category10();

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");

var line = d3.svg.line()
    .interpolate("basis")
    .x(function(d) { return x(d.date); })
    .y(function(d) { return y(d.temperature); });

var svg = d3.select("body").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
       
var my_graph = function(error, data) {
  if (error) throw error;
    
  color.domain(d3.keys(data[0]).filter(function(key) { return key !== "date"; }));

  data.forEach(function(d) {
    d.date = parseDate(d.date);
  });

  var cities = color.domain().map(function(name) {
    return {
      name: name,
      values: data.map(function(d) {
        return {date: d.date, temperature: +d[name]};
      })
    };
  });

  x.domain(d3.extent(data, function(d) { return d.date; }));

  y.domain([
    d3.min(cities, function(c) { return d3.min(c.values, function(v) { return v.temperature; }); }),
    d3.max(cities, function(c) { return d3.max(c.values, function(v) { return v.temperature; }); })
  ]);

  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis);

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 6)
      .attr("dy", ".71em")
      .style("text-anchor", "end")
      .text("Temperature (ºC)");

  var city = svg.selectAll(".city")
      .data(cities)
    .enter().append("g")
      .attr("class", "city");

  city.append("path")
      .attr("class", "line")
      .attr("d", function(d) { return line(d.values); })
      .style("stroke", function(d) { return color(d.name); });

  city.append("text")
      .datum(function(d) { return {name: d.name, value: d.values[d.values.length - 1]}; })
      .attr("transform", function(d) { return "translate(" + x(d.value.date) + "," + y(d.value.temperature) + ")"; })
      .attr("x", 3)
      .attr("dy", ".35em")
      .text(function(d) { return d.name; });
};

data = {data}
my_graph("",data)


</script>


Overwriting line_chart.template


In [3]:
from IPython.display import IFrame

In [10]:
# Lectura de los datos
import sqlite3
conn = sqlite3.connect("D:/master/Master-in-Data-Science/Proyecto/Datos/worldTimeSeries_mini.sqlite")
cursor = conn.cursor()

station = 'ITE00100554'
result = []

# Se calcula la temperatura media por cada uno de los meses
rows = cursor.execute("""select 
                            substr(date,1,6) as date, 
                            avg(TMAX) as TMAX, 
                            avg(TMIN) as TMIN, 
                            avg(TAVG) as TAVG
                        from world_time 
                        group by substr(date,1,6);""")
#                        order by substr(date,1,6);""")
#                        where STATION = '"""+station+""""' 

n = 0
for row in rows:
    n = n + 1
    date = str(row[0])+"01"
    
    dmax = row[1]
    if dmax == None:
        dmax = 0
        
    dmin = row[2]
    if dmin == None:
        dmin = 0
        
    davg = row[3]
    if davg == None:
        davg = 0
        
    result.append({"date":date,"MAX":dmax,"MIN":dmin,"AVG":davg})

print(n)
conn.close()

# Se lee la plantilla
input= open("line_chart.template")
text = input.read()
input.close()

# Se reemplazan los datos
text=text.replace("{data}",str(result))

# Se escribe el html
output = open("line_chart.html","w")
output.write(text)
output.close()

IFrame("line_chart.html",800,800)


2952


In [ ]:
%%writefile line_chart.template
<meta charset="utf-8">
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1q8mTJOASx8j1Au+a5WDVnPi2lkFfwwEAa8hDDdjZlpLegxhjVME1fgjWPGmkzs7" crossorigin="anonymous">

<style>

body {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.line {
  fill: none;
  stroke: steelblue;
  stroke-width: 1.5px;
}

</style>

<body>

    <div class="container">
        <div class="row">
            <div id="entrada_datos" class="col-md-6">
                <h4>Selecciona los parametros a visualizar</h3>
                
            </div>
            <div id="chart2" class="col-md-6">
                <h2>Temperatura en Paris</h2>
          </div>
        </div>
    </div>

<script src="//d3js.org/d3.v3.min.js"></script>
<script>

var margin = {top: 20, right: 20, bottom: 30, left: 50},
    width = {width} - margin.left - margin.right,
    height = {height} - margin.top - margin.bottom;

var formatDate = d3.time.format("%Y%m%d");

var x = d3.time.scale()
    .range([0, width]);

var y = d3.scale.linear()
    .range([height, 0]);

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");

var line = d3.svg.line()
    .x(function(d) { return x(d.date); })
    .y(function(d) { return y(d.close); });
    
var process = function(error, data, element) {
  if (error) throw error;
    
    data = data.map(type)

    var svg = d3.select(element).append("svg")
        .attr("width", width + margin.left + margin.right)
        .attr("height", height + margin.top + margin.bottom)
        .append("g")
        .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
    
     x.domain(d3.extent(data, function(d) { return d.date; }));
     y.domain(d3.extent(data, function(d) { return d.close; }));

  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis);

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 6)
      .attr("dy", ".71em")
      .style("text-anchor", "end")
      .text("Temperature");

  svg.append("path")
      .datum(data)
      .attr("class", "line")
      .attr("d", line);
}
    

var data = {data}
process("",data,"#chart1")

var data2 = {data2}
process("",data2,"#chart2")

function type(d) {
  d.date = formatDate.parse(""+d.date);
  d.close = +d.close;
  return d;
}

</script>